Himmelblau e sua função
========================================



## Introdução



O probelma escolhido foi para tentar descobrir o mínimo global da função de Himmelblau.

## Objetivo



Encontre a coordenada $(x,y)$ do mínimo global da função de Himmelblau abaixo.

$$
f(x,y) = (x^2 +y -11)^2 + (x + y^2 - 7)^2
$$

## Importações



Importanto todas as biblitecas nescessarias e funções.

Para esse desafio utilizaremos funções criadas no [arquivo de funções](funcoes.py), e as bibliotecas random e numpy.

In [1]:
from funcoes import população_caixas_n_binarias
from funcoes import função_objetivo_Rastrigin_pop
from funcoes import função_objetivo_Himmelblau_pop as func_objetivo_pop
from funcoes import selecao_torneio_min as fun_selecao
from funcoes import cruzamento_ponto_simples as fun_cruzamento
from funcoes import mutação_Função as mutacao
from random import random
import numpy as np
import plotly.express as px

## Códigos e discussão



Primeiro precisamos definir as constantes que serão utilizadas, elas seram os parametros do nosso algoritimo genetico.



In [2]:
VALOR_MAXIMO_CAIXA = 6
VALOR_MINIMO_CAIXA = -6
TAMANHO_POP = 100
NUM_GENES = 2
NUM_GERAÇOES = 100000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTAÇÃO = 0.8

Agora para facilitar a utilização do codigo [Caixa não binária](experimento%20A.04%20-%20caixas%20nao-binarias.ipynb), que é o codigo que iremos nos basear, vamos criar funções locais, para não mudar tanto a logica do codigo.


In [3]:
# funções locais


def população_inicial(tamanho, n_gene):
    return população_caixas_n_binarias(
        VALOR_MAXIMO_CAIXA, tamanho, n_gene, VALOR_MINIMO_CAIXA, False
    )


def fun_mutação(individuo):
    # return mutacao_cnb(VALOR_MAXIMO_CAIXA, individuo, VALOR_MINIMO_CAIXA, False)
    return mutacao(individuo,VALOR_MAXIMO_CAIXA,VALOR_MINIMO_CAIXA)

Agora podemos escrever o codigo que será nosso algoritimo genético.

In [4]:
população_total = list()

população = população_inicial(TAMANHO_POP, NUM_GENES)
melhor_fitness_ja_visto = float("inf")

população_total.append(população)


geração = 0
melhores_geracoes = []


while round(melhor_fitness_ja_visto, 5) > 0 or NUM_GENES >= geração:
    geração += 1
    fitnes = func_objetivo_pop(população)
    população = fun_selecao(população, fitnes)

    pais = população[0::2]
    maes = população[1::2]
    contador = 0

    for pai, mae in zip(pais, maes):
        if random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = fun_cruzamento(pai, mae)
            população[contador] = filho1
            população[contador + 1] = filho2
        contador += 2

    for index in range(len(população)):
        if random() <= CHANCE_MUTAÇÃO:
            população[index] = fun_mutação(população[index])

    fitness = func_objetivo_pop(população)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:
        melhores_geracoes.append(geração)
        população_total.append(população)
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = população[posicao]
        melhor_fitness_ja_visto = menor_fitness
        print(
            f"geração:{geração} - melhor individuo {melhor_individuo_ja_visto} - fitness: {melhor_fitness_ja_visto}"
        )


geração:1 - melhor individuo [3.118181950756629, 1.4604404029237816] - fitness: 3.092434629221012
geração:2 - melhor individuo [-3.7715890676699972, -3.462877693999648] - fitness: 1.544877125968035
geração:3 - melhor individuo [2.872815260070377, 2.299174199207039] - fitness: 1.5438084845668394
geração:4 - melhor individuo [2.9251817819936456, 2.299174199207039] - fitness: 1.488226236828628
geração:5 - melhor individuo [2.873564501847926, 2.150786028104911] - fitness: 0.5997211481197349
geração:6 - melhor individuo [2.8778245524008925, 2.1051458716132894] - fitness: 0.4715122258983
geração:7 - melhor individuo [2.959706816468772, 2.141832810775524] - fitness: 0.3090415932428231
geração:8 - melhor individuo [2.959706816468772, 2.1057994471990433] - fitness: 0.17335952607348717
geração:10 - melhor individuo [2.959706816468772, 2.1053491207195854] - fitness: 0.1719895865408121
geração:12 - melhor individuo [2.990064372164196, 1.9906607084125387] - fitness: 0.006969286409420922
geração:17 

Algumas mudanças foram aplicadas a este codigo para melhorar o desenpenho, e poder utiliza-lo depois para mostrar visualmente o que acontece.

1. A criterio de parada foi quando o menor fitines, que nesse caso seria o minimo da função, for menor a 0, com tolerancia de 3 casas decimais, ou quando o numero de gerações forem maiores que `NUM_GERAÇOES`.

2. A função objetivo foi alterada de uma simples soma para a função de Himmelblau.

3. Quando o individou é escolhido para ser mutado pode acontecer 4 casos, dependendo da escolha aleatoria dentro da função de mutação:
    + Fina: Anda apenas uma cordenada para mais ou para menos até 0.1% do valor da mesma, tem maior chance de acontecer.
    + Media: Anda apenas uma cordenada para mais ou para menos até 10% do valor da mesma, segunda maior chance de acontecer.
    + Grossa: Anda apenas uma cordenada para mais ou para menos até 100% do valor da mesma, terceira maior chance de acontecer.
    + Aleatoria: Escolhe um novo valor para a cordenada, dentro do range que o usuario escolheu, tem menor chance de acontecer.
    
    Cada tipo de mutação é por um motivo, a maior para encontar um minimo melhor, 
    
4. Foi utilizado a função de torneio de minimização para selecionar os melhores individuos.
    

In [5]:
# Sua lista de listas de pontos
lista_de_listas = []
for populacão_geração, geração in zip(população_total, melhores_geracoes):
    lista_geração = []
    for individuo in populacão_geração:
        x = individuo[0]
        y = individuo[1]
        z = (x**2 + y - 11) ** 2 + (x + y**2 - 7) ** 2
        g = geração
        lista_geração.append((x, y, z, g))
    lista_de_listas.append(lista_geração)
# Criar um dataframe com todas as listas de pontos
data = []
for lista in lista_de_listas:
    for ponto in lista:
        data.append({"x": ponto[0], "y": ponto[1], "z": ponto[2], "Geração": ponto[3]})

x_vals = np.linspace(-6, 6)
y_vals = np.linspace(-6, 6)

x_vals, y_vals = np.meshgrid(x_vals, y_vals)
z_vals = (x_vals**2 + y_vals - 11) ** 2 + (x_vals + y_vals**2 - 7) ** 2
# Criar o gráfico 3D usando Plotly Express
fig = px.scatter_3d(
    data,
    x="x",
    y="y",
    z="z",
    animation_frame="Geração",
    color_discrete_sequence=["red"],
)

fig.add_surface(x=x_vals, y=y_vals, z=z_vals, colorscale="viridis")

# Aumentar o tamanho do gráfico
fig.update_layout(width=800, height=600)

# Exibir o gráfico 3D
fig.show()

Com esse plot podemos visualizar o que aconteceu em cada geração, que teve um `menor fitness`, podemos perceber as 3 mutações, e a rapidez que os individuos convergem para o minimo global.

Agora para testar com um desafio maior irei testar o mesmo algoritimo, mudando somente a função objetivo para a função Rastrigin.
$$f(x,y)=20 + \left(x^2 - 10cos(2πx)\right) + \left(y^2 - 10cos(2πy)\right)$$
Essa função possue varios minimos locais e apenas um minimo global, na cordenada $f(0,0)=0$.

In [6]:
população_total = []

população = população_inicial(TAMANHO_POP, NUM_GENES)
melhor_fitness_ja_visto = float("inf")

população_total.append(população)


geração = 0
melhores_geracoes = []


while round(melhor_fitness_ja_visto,5) > 0 or NUM_GENES >= geração:
    geração += 1
    fitnes = função_objetivo_Rastrigin_pop(população)
    população = fun_selecao(população, fitnes)

    pais = população[0::2]
    maes = população[1::2]
    contador = 0

    for pai, mae in zip(pais, maes):
        if random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = fun_cruzamento(pai, mae)
            população[contador] = filho1
            população[contador + 1] = filho2
        contador += 2

    for index in range(len(população)):
        if random() <= CHANCE_MUTAÇÃO:
            população[index] = fun_mutação(população[index])

    fitness = função_objetivo_Rastrigin_pop(população)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:
        melhores_geracoes.append(geração)
        população_total.append(população)
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = população[posicao]
        melhor_fitness_ja_visto = menor_fitness
        print(
            f"geração:{geração} - melhor individuo {melhor_individuo_ja_visto} - fitness: {melhor_fitness_ja_visto}"
        )


geração:1 - melhor individuo [1.9803737603262221, 1.0304225006892356] - fitness: 5.2417241921137165
geração:2 - melhor individuo [0.9808731733717585, -0.003942059140936682] - fitness: 1.0373211671935625
geração:4 - melhor individuo [0.025657381113692573, -0.004280137986980066] - fitness: 0.13395481476810644
geração:5 - melhor individuo [0.02508259169698509, -0.004280137986980066] - fitness: 0.12819310170651832
geração:6 - melhor individuo [0.023179183137452184, -0.004276966624902808] - fitness: 0.11003255068726148
geração:7 - melhor individuo [0.0037385251668972746, -0.004095437976482358] - fitness: 0.006100085012432643
geração:9 - melhor individuo [0.0013627063341766523, -0.0037043723113476773] - fitness: 0.0030906938226991088
geração:10 - melhor individuo [0.0013627063341766523, -0.0037017641294769268] - fitness: 0.0030868619158699317
geração:11 - melhor individuo [0.0033311171366449924, -0.001150393104535394] - fitness: 0.0024638981814977257
geração:12 - melhor individuo [0.00333111

Podemos observar que foi melhorando o fitnees, e foi pulando de minimos locais até chegar no global.

In [7]:
# Sua lista de listas de pontos
lista_de_listas = []
for populacão_geração, geração in zip(população_total, melhores_geracoes):
    lista_geração = []
    for individuo in populacão_geração:
        x = individuo[0]
        y = individuo[1]
        z = 20 + (x**2 - 10 * np.cos(2 * np.pi * x)) + (y**2 - 10 * np.cos(2 * np.pi * y))
        g = geração
        lista_geração.append((x, y, z, g))
    lista_de_listas.append(lista_geração)
# Criar um dataframe com todas as listas de pontos
data = []
for lista in lista_de_listas:
    for ponto in lista:
        data.append({"x": ponto[0], "y": ponto[1], "z": ponto[2], "Geração": ponto[3]})

x_vals = np.linspace(-6, 6)
y_vals = np.linspace(-6, 6)

x_vals, y_vals = np.meshgrid(x_vals, y_vals)
z_vals = 20 + (x_vals**2 - 10 * np.cos(2 * np.pi * x_vals)) + (y_vals**2 - 10 * np.cos(2 * np.pi * y_vals))

# Criar o gráfico 3D usando Plotly Express
fig = px.scatter_3d(
    data,
    x="x",
    y="y",
    z="z",
    animation_frame="Geração",
    color_discrete_sequence=["red"],
)

fig.add_surface(x=x_vals, y=y_vals, z=z_vals, colorscale="viridis")

# Aumentar o tamanho do gráfico
fig.update_layout(width=800, height=600)

# Exibir o gráfico 3D
fig.show()

Para ficar mais fácil a visualização tem o plot interativo de cada geração.

## Conclusão



O codigo criado é eficiente para descobrir o minimo global dessas funções, conseguindo realizar em poucas gerações e pouco tempo.

Isso se dá principalmente pela mutação, que pode occorer em 4 passo, sendo essas fina, media, grossa e aleatoria, assim o algoritimo pode explorar, andar e caminar na superficie encontrando o ponto de minimo rapidamente, logico que utilizando como base esses exemplos.

Outra coisa interressante nesse algoritimo que podemos perceber que mesmo sem uma real logica de mutação,como o gradiente, portanto sendo apenas aleatoria, o algoritimo pode rapidamente chegar ao ponto de minimo global.

E utilizando os plots podemos visualizar como o algoritimo "pensa", como é gerado cada ponto, lembrando que está sendo mostrado apenas as gerações que obteve um valor de fitiness menor.

## Referências consultadas



[1] Himmelblau’s function. In: Wikipedia. [s.l.: s.n.], 2022. Disponível em: <https://en.wikipedia.org/w/index.php?title=Himmelblau%27s_function&oldid=1069627492>. Acesso em: 15 jun. 2023.

[2] Rastrigin function. In: Wikipedia. [s.l.: s.n.], 2023. Disponível em: <https://en.wikipedia.org/w/index.php?title=Rastrigin_function&oldid=1134743937>. Acesso em: 15 jun. 2023.


## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.

